In [ ]:
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
# Load labels.csv
labels_df = pd.read_csv('labels.csv')  # Update with your file path

# Load list_bbox_celeba.txt
bbox_df = pd.read_csv('list_bbox_celeba.txt', delim_whitespace=True, header=1)  # Update with your file path

# Merge dataframes based on the 'filename' column
merged_df = labels_df.merge(bbox_df, on='filename')


In [ ]:
merged_df['x_normalized'] = merged_df['x_1'] / merged_df['width']
merged_df['y_normalized'] = merged_df['y_1'] / merged_df['height']
merged_df['width_normalized'] = merged_df['width'] / merged_df['width']
merged_df['height_normalized'] = merged_df['height'] / merged_df['height']


In [ ]:
class CustomImageDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, dataframe, batch_size, image_size, class_names, bbox_columns):
        self.dataframe = dataframe
        self.batch_size = batch_size
        self.image_size = image_size
        self.class_names = class_names
        self.bbox_columns = bbox_columns
        self.num_classes = len(class_names)
        self.indexes = np.arange(len(dataframe))

    def __len__(self):
        return int(np.ceil(len(self.dataframe) / self.batch_size))

    def __getitem__(self, index):
        batch_indexes = self.indexes[index * self.batch_size: (index + 1) * self.batch_size]
        batch_data = self.dataframe.iloc[batch_indexes]

        batch_images = []
        batch_classes = []
        batch_bboxes = []

        for _, row in batch_data.iterrows():
            image = cv2.imread(row['filename'])  # Load image using OpenCV
            image = cv2.resize(image, self.image_size)  # Resize to desired size

            bbox_normalized = [row[col] for col in self.bbox_columns]  # Get normalized bbox coordinates

            class_vector = [row[class_name] for class_name in self.class_names]  # Get class labels

            batch_images.append(image)
            batch_classes.append(class_vector)
            batch_bboxes.append(bbox_normalized)

        return (
            np.array(batch_images),
            {'class_names': np.array(batch_classes), 'bboxes': np.array(batch_bboxes)}
        )

# List of class names
class_names = labels_df.columns[1:].tolist()

# List of bbox columns
bbox_columns = ['x_normalized', 'y_normalized', 'width_normalized', 'height_normalized']

# Image size
image_size = (224, 224)

# Create data generator
batch_size = 32
data_generator = CustomImageDataGenerator(merged_df, batch_size, image_size, class_names, bbox_columns)


In [ ]:
data_flow = data_generator.flow_from_dataframe(
    merged_df,
    x_col='filename',
    y_col={'class_names': class_names, 'bboxes': bbox_columns},
    batch_size=batch_size,
    target_size=image_size,
    class_mode='raw',  # We will use 'raw' mode to pass custom target data
)


In [ ]:
def preprocess_images_and_bboxes(images, bboxes, target_size):
    cropped_images = []
    for image, bbox in zip(images, bboxes):
        x, y, width, height = bbox
        x_min = int(x * image.shape[1])
        y_min = int(y * image.shape[0])
        x_max = int((x + width) * image.shape[1])
        y_max = int((y + height) * image.shape[0])
        
        cropped_image = image[y_min:y_max, x_min:x_max]
        resized_image = cv2.resize(cropped_image, target_size)
        
        cropped_images.append(resized_image)
    
    return np.array(cropped_images)


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, Flatten, Dense, GlobalAveragePooling2D

base_model = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

roi_input = Input(shape=(None, None, 3))  # Shape can be adjusted based on your bounding box sizes
roi_features = base_model(roi_input)
global_avg_pooling = GlobalAveragePooling2D()(roi_features)
class_output = Dense(num_classes, activation='softmax')(global_avg_pooling)
bbox_output = Dense(4, activation='linear')(global_avg_pooling)

model = Model(inputs=roi_input, outputs=[class_output, bbox_output])


In [ ]:
model.compile(
    optimizer='adam',
    loss={'class_output': 'categorical_crossentropy', 'bbox_output': 'mean_squared_error'},
    loss_weights={'class_output': 1.0, 'bbox_output': 1.0},
    metrics=['accuracy']
)


In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    for batch_images, batch_targets in data_flow:
        cropped_images = preprocess_images_and_bboxes(batch_images, batch_targets['bboxes'], target_size=(224, 224))
        
        class_targets = batch_targets['class_names']
        bbox_targets = batch_targets['bboxes']
        
        model.fit(
            cropped_images,
            {'class_output': class_targets, 'bbox_output': bbox_targets},
            batch_size=batch_size
        )
